<a href="https://colab.research.google.com/github/JellePiepenbrock/neurosmash/blob/master/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
from IPython.display import clear_output
import os, sys

drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/projects/entity_tagging/')

base_url = '/content//gdrive/My Drive/projects/NIPS/neurosmash'

clear_output()

## Beta VAE

In [0]:
"""model.py"""

import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable


def reparametrize(mu, logvar):
    std = logvar.div(2).exp()
    eps = Variable(std.data.new(std.size()).normal_())
    return mu + std*eps


class View(nn.Module):
    def __init__(self, size):
        super(View, self).__init__()
        self.size = size

    def forward(self, tensor):
        return tensor.view(self.size)


class BetaVAE_H(nn.Module):
    """Model proposed in original beta-VAE paper(Higgins et al, ICLR, 2017)."""

    def __init__(self, z_dim=10, nc=3):
        super(BetaVAE_H, self).__init__()
        self.z_dim = z_dim
        self.nc = nc
        self.encoder = nn.Sequential(
            nn.Conv2d(nc, 32, 4, 2, 1),          # B,  32, 32, 32
            nn.ReLU(True),
            nn.Conv2d(32, 32, 4, 2, 1),          # B,  32, 16, 16
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, 2, 1),          # B,  64,  8,  8
            nn.ReLU(True),
            nn.Conv2d(64, 64, 4, 2, 1),          # B,  64,  4,  4
            nn.ReLU(True),
            nn.Conv2d(64, 256, 4, 1),            # B, 256,  1,  1
            nn.ReLU(True),
            View((-1, 256*1*1)),                 # B, 256
            nn.Linear(256, z_dim*2),             # B, z_dim*2
        )
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),               # B, 256
            View((-1, 256, 1, 1)),               # B, 256,  1,  1
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 64, 4),      # B,  64,  4,  4
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 64, 4, 2, 1), # B,  64,  8,  8
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), # B,  32, 16, 16
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32, 32, 32
            nn.ReLU(True),
            nn.ConvTranspose2d(32, nc, 4, 2, 1),  # B, nc, 64, 64
        )

        self.weight_init()

    def weight_init(self):
        for block in self._modules:
            for m in self._modules[block]:
                kaiming_init(m)

    def forward(self, x):
        distributions = self._encode(x)
        mu = distributions[:, :self.z_dim]
        logvar = distributions[:, self.z_dim:]
        z = reparametrize(mu, logvar)
        x_recon = self._decode(z)

        return x_recon, mu, logvar

    def _encode(self, x):
        return self.encoder(x)

    def _decode(self, z):
        return self.decoder(z)


class BetaVAE_B(BetaVAE_H):
    """Model proposed in understanding beta-VAE paper(Burgess et al, arxiv:1804.03599, 2018)."""

    def __init__(self, z_dim=10, nc=1):
        super(BetaVAE_B, self).__init__()
        self.nc = nc
        self.z_dim = z_dim

        self.encoder = nn.Sequential(
            nn.Conv2d(nc, 32, 4, 2, 1),          # B,  32, 32, 32
            nn.ReLU(True),
            nn.Conv2d(32, 32, 4, 2, 1),          # B,  32, 16, 16
            nn.ReLU(True),
            nn.Conv2d(32, 32, 4, 2, 1),          # B,  32,  8,  8
            nn.ReLU(True),
            nn.Conv2d(32, 32, 4, 2, 1),          # B,  32,  4,  4
            nn.ReLU(True),
            View((-1, 32*4*4)),                  # B, 512
            nn.Linear(32*4*4, 256),              # B, 256
            nn.ReLU(True),
            nn.Linear(256, 256),                 # B, 256
            nn.ReLU(True),
            nn.Linear(256, z_dim*2),             # B, z_dim*2
        )

        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),               # B, 256
            nn.ReLU(True),
            nn.Linear(256, 256),                 # B, 256
            nn.ReLU(True),
            nn.Linear(256, 32*4*4),              # B, 512
            nn.ReLU(True),
            View((-1, 32, 4, 4)),                # B,  32,  4,  4
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32,  8,  8
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32, 16, 16
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32, 32, 32
            nn.ReLU(True),
            nn.ConvTranspose2d(32, nc, 4, 2, 1), # B,  nc, 64, 64
        )
        self.weight_init()

    def weight_init(self):
        for block in self._modules:
            for m in self._modules[block]:
                normal_init(m)

    def forward(self, x):
        distributions = self._encode(x)
        mu = distributions[:, :self.z_dim]
        logvar = distributions[:, self.z_dim:]
        z = reparametrize(mu, logvar)
        x_recon = self._decode(z).view(x.size())

        return x_recon, mu, logvar

    def _encode(self, x):
        return self.encoder(x)

    def _decode(self, z):
        return self.decoder(z)


def kaiming_init(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        init.kaiming_normal(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
        m.weight.data.fill_(1)
        if m.bias is not None:
            m.bias.data.fill_(0)


def normal_init(m, mean, std):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        m.weight.data.normal_(mean, std)
        if m.bias.data is not None:
            m.bias.data.zero_()
    elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
        m.weight.data.fill_(1)
        if m.bias.data is not None:
            m.bias.data.zero_()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim import Adam
from PIL import Image
import torch.nn.functional as F
import numpy as np
import torch
from torch.autograd import Variable
from torch.distributions import Categorical
from random import randint

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import io, transform


%matplotlib inline

from torchvision.utils import save_image
device = torch.device('cuda')

batch_size = 64

objective = 'H'
beta = 10
lr =  1e-4
z_dim =  32 
max_iter =  1.5e6
decoder_dist = 'gaussian'

vae = BetaVAE_B(nc=3, z_dim=z_dim) if objective == 'B' else BetaVAE_H(nc=3)
vae = vae.to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=lr, betas=(0.9, 0.999))
print('Loaded model.')

# https://github.com/1Konny/Beta-VAE/blob/master/solver.py 


def reconstruction_loss(x, x_recon, distribution):
    batch_size = x.size(0)
    assert batch_size != 0

    if distribution == 'bernoulli':
        recon_loss = F.binary_cross_entropy_with_logits(x_recon, x, size_average=False).div(batch_size)
    elif distribution == 'gaussian':
        x_recon = F.sigmoid(x_recon)
        recon_loss = F.mse_loss(x_recon, x, size_average=False).div(batch_size)
    else:
        recon_loss = None

    return recon_loss


def kl_divergence(mu, logvar):
    batch_size = mu.size(0)
    assert batch_size != 0
    if mu.data.ndimension() == 4:
        mu = mu.view(mu.size(0), mu.size(1))
    if logvar.data.ndimension() == 4:
        logvar = logvar.view(logvar.size(0), logvar.size(1))

    klds = -0.5*(1 + logvar - mu.pow(2) - logvar.exp())
    total_kld = klds.sum(1).mean(0, True)
    dimension_wise_kld = klds.mean(0)
    mean_kld = klds.mean(1).mean(0, True)

    return total_kld, dimension_wise_kld, mean_kld


def compare(x):
    recon_x, _, _ = vae(x)
    return torch.cat([x, recon_x])

def main(episodes):
    dataset = torch.load('{}/file.pt'.format(base_url)) / 255
    # dataset = torch.from_numpy(transform.resize(dataset, (len(dataset), 32, 3, 32, 32)))
    # print(dataset.shape)
    dataloader = torch.utils.data.DataLoader(dataset.reshape(32*len(dataset), 3, 64, 64), batch_size=batch_size, shuffle=True)
    epochs = 500
    
    for epoch in range(epochs):
        for x in dataloader:
            x = x.to(device)
            x_recon, mu, logvar = vae(x)
            recon_loss = reconstruction_loss(x, x_recon, decoder_dist)
            total_kld, dim_wise_kld, mean_kld = kl_divergence(mu, logvar)

            if objective == 'H':
                beta_vae_loss = recon_loss + beta*total_kld
            # elif objective == 'B':
            #     C = torch.clamp(self.C_max/self.C_stop_iter*self.global_iter, 0, self.C_max.data[0])
            #     beta_vae_loss = recon_loss + self.gamma*(total_kld-C).abs()
            
            optimizer.zero_grad()
            beta_vae_loss.backward()
            optimizer.step()
        if epoch == 100:
            for param_group in optimizer.param_groups:
                param_group['lr'] = 1e-5
        # Store one image per epoch to showcase development of VAE.
        if (epoch % 10 == 0) or (epoch == epochs-1):
            fixed_x = dataset[0][10].unsqueeze(0)
            compare_x = compare(fixed_x.to(device))
            save_image(compare_x.data.cpu(), '{}/images_vaeb/sample_image_epoch_{}.png'.format(base_url, epoch+1))

        print('Epoch {}, loss {}'.format(epoch+1, recon_loss.item()))
    # Store model. 
    torch.save(vae.state_dict(), 'vae.torch')

    # Show final result. 
    img = mpimg.imread('{}/images_vaeb/sample_image_epoch_{}.png'.format(base_url, epochs))
    imgplot = plt.imshow(img)
    plt.show()

main(50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:150: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Loaded model.


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch 1, loss 223.96868896484375
Epoch 2, loss 97.2720947265625
Epoch 3, loss 53.61050796508789
Epoch 4, loss 36.82554626464844
Epoch 5, loss 32.377281188964844
Epoch 6, loss 26.23065948486328


## InfoVAE

In [0]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
    
class Reshape(torch.nn.Module):
    def __init__(self, outer_shape):
        super(Reshape, self).__init__()
        self.outer_shape = outer_shape
    def forward(self, x):
        return x.view(x.size(0), *self.outer_shape)

# Encoder and decoder use the DC-GAN architecture
class Encoder(torch.nn.Module):
    def __init__(self, z_dim):
        super(Encoder, self).__init__()
        self.model = torch.nn.ModuleList([
            torch.nn.Conv2d(3, 64, 4, 2, padding=1),
            torch.nn.LeakyReLU(),
            torch.nn.Conv2d(64, 128, 4, 2, padding=1),
            torch.nn.LeakyReLU(),
            Flatten(),
            torch.nn.Linear(32768, 1024),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(1024, z_dim)
        ])
        
    def forward(self, x):
        #print("Encoder")
        #print(x.size())
        for layer in self.model:
            x = layer(x)
            #print(x.size())
        return x
    
    
class Decoder(torch.nn.Module):
    def __init__(self, z_dim):
        super(Decoder, self).__init__()
        self.model = torch.nn.ModuleList([
            torch.nn.Linear(z_dim, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 32768),
            torch.nn.ReLU(),
            Reshape((128,16,16,)),
            torch.nn.ConvTranspose2d(128, 64, 4, 2, padding=1),
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(64, 3, 4, 2, padding=1),
            torch.nn.Sigmoid()
        ])
        
    def forward(self, x):
        #print("Decoder")
        #print(x.size())
        for layer in self.model:
            x = layer(x)
            #print(x.size())
        return x

def compute_kernel(x, y):
    x_size = x.size(0)
    y_size = y.size(0)
    dim = x.size(1)
    x = x.unsqueeze(1) # (x_size, 1, dim)
    y = y.unsqueeze(0) # (1, y_size, dim)
    tiled_x = x.expand(x_size, y_size, dim)
    tiled_y = y.expand(x_size, y_size, dim)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input) # (x_size, y_size)

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

class Model(torch.nn.Module):
    def __init__(self, z_dim):
        super(Model, self).__init__()
        self.encoder = Encoder(z_dim)
        self.decoder = Decoder(z_dim)
        
    def forward(self, x):
        z = self.encoder(x)
        x_reconstructed = self.decoder(z)
        return z, x_reconstructed

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math
import matplotlib.pyplot as plt


class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

%matplotlib inline
z_dim = 4
vae = Model(z_dim).to(device)#VAE(image_channels=3).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
print('Loaded model.')

# https://github.com/napsternxg/pytorch-practice/blob/master/Pytorch%20-%20MMD%20VAE.ipynb
def compare(x):
    _, recon_x = vae(x)
    return torch.cat([x, recon_x])


def main(episodes):
    dataset = torch.load('{}/file.pt'.format(base_url))[:10] / 255
    batch_size = 256
    dataloader = torch.utils.data.DataLoader(dataset.reshape(32*len(dataset), 3, 64, 64), batch_size=batch_size, shuffle=True)
    epochs = 100
    for epoch in range(epochs):
        losses = [] 
        for i, b in enumerate(dataloader):
            optimizer.zero_grad()
            x = b.to(device)
            z, x_reconstructed = vae(x) 

            true_samples = torch.randn(200, z_dim, requires_grad=False).to(device)
            mmd = compute_mmd(true_samples, z)
            nll = (x_reconstructed - x).pow(2).mean()
            loss = nll + mmd*2
            loss.backward()
            optimizer.step()

        # Store one image per epoch to showcase development of VAE.
        print('Epoch {} loss {}'.format(epoch, loss.item()))
        fixed_x = dataset[1][30].unsqueeze(0)
        compare_x = compare(fixed_x.to(device))
        save_image(compare_x.data.cpu(), '{}/images/sample_image_epoch_{}.png'.format(base_url, epoch+1))

        print('Epoch {}, loss {}'.format(epoch+1, loss.item()))
    # Store model. 
    torch.save(vae.state_dict(), '{}/vae.torch'.format(base_url))

    # Show final result. 
    img = mpimg.imread('{}/images/sample_image_epoch_{}.png'.format(base_url, epochs))
    imgplot = plt.imshow(img)
    plt.show()

main(50)

Loaded model.
Epoch 0 loss 3.2989015579223633
(5, 64, 64, 1)
(5, 64, 64, 1)


ValueError: ignored

## VAE

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class UnFlatten(nn.Module):
    def forward(self, input, size=1024):
        return input.view(input.size(0), size, 1, 1)

class VAE(nn.Module):
    # def __init__(self, image_channels=3, h_dim=1024, z_dim=64):
    #     super(VAE, self).__init__()
    #     self.encoder = nn.Sequential(
    #         nn.Conv2d(image_channels, 4, kernel_size=4, stride=2),
    #         nn.ReLU(),
    #         nn.Conv2d(4, 4, kernel_size=4, stride=2),
    #         nn.ReLU(),
    #         nn.Conv2d(4, 4, kernel_size=4, stride=2),
    #         nn.ReLU(),
    #         nn.Conv2d(4, 4, kernel_size=4, stride=2),
    #         nn.ReLU(),
    #         Flatten()
    #     )
    #     self.fc1 = nn.Linear(h_dim, z_dim)
    #     self.fc2 = nn.Linear(h_dim, z_dim)
    #     self.fc3 = nn.Linear(z_dim, h_dim)
        
    #     self.decoder = nn.Sequential(
    #         UnFlatten(),
    #         nn.ConvTranspose2d(h_dim, 4, kernel_size=8, stride=2),
    #         nn.ReLU(),
    #         nn.ConvTranspose2d(4, 4, kernel_size=7, stride=2),
    #         nn.ReLU(),
    #         nn.ConvTranspose2d(4, 4, kernel_size=5, stride=2),
    #         nn.ReLU(),
    #         nn.ConvTranspose2d(4, 4, kernel_size=6, stride=2),
    #         nn.ReLU(),
    #         nn.ConvTranspose2d(4, 4, kernel_size=5, stride=2),
    #         nn.ReLU(),
    #         nn.ConvTranspose2d(4, 4, kernel_size=3, stride=2),
    #         nn.ReLU(),
    #         nn.ConvTranspose2d(4, image_channels, kernel_size=4, stride=2),
    #         nn.Sigmoid(),
    #     )
    def __init__(self, image_channels=3, h_dim=1024, z_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size()).to(device)
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = self.decoder(z)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar

In [26]:
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim import Adam
from PIL import Image
import torch.nn.functional as F
import numpy as np
import torch
from torch.autograd import Variable
from torch.distributions import Categorical
from random import randint

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from torchvision.utils import save_image
device = torch.device('cuda')

batch_size = 32

vae = VAE(image_channels=3).to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
print('Loaded model.')

# https://github.com/sksq96/pytorch-vae/blob/master/vae-cnn.ipynb <-- source for code. 
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) #torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD

def compare(x):
    recon_x, _, _ = vae(x)
    return torch.cat([x, recon_x])

def main(episodes):
    dataset = torch.load('{}/file.pt'.format(base_url)) / 255
    dataloader = torch.utils.data.DataLoader(dataset.reshape(32*len(dataset), 3, 64, 64), batch_size=batch_size, shuffle=True)
    epochs = 100
    for epoch in range(epochs):
        losses = [] 
        for i, b in enumerate(dataloader):
            optimizer.zero_grad()
            b = b.to(device)
            recon_images, mu, logvar = vae(b)
            loss, bce, kld = loss_fn(recon_images, b, mu, logvar)
            loss.backward()
            optimizer.step()

        # Store one image per epoch to showcase development of VAE.
        if (epoch % 50 == 0) or (epoch == epochs-1):
            fixed_x = dataset[0][10].unsqueeze(0)
            compare_x = compare(fixed_x.to(device))
            save_image(compare_x.data.cpu(), '{}/images/sample_image_epoch_{}.png'.format(base_url, epoch+1))

        print('Epoch {}, loss {}'.format(epoch+1, loss.item()))
    # Store model. 
    torch.save(vae.state_dict(), 'vae.torch')

    # Show final result. 
    img = mpimg.imread('{}/images/sample_image_epoch_{}.png'.format(base_url, epochs))
    imgplot = plt.imshow(img)
    plt.show()

main(50)

Loaded model.
Epoch 1, loss 2230.425537109375
Epoch 2, loss 780.3670654296875
Epoch 3, loss 381.1385498046875
Epoch 4, loss 259.9405822753906
Epoch 5, loss 203.65121459960938
Epoch 6, loss 181.8278350830078
Epoch 7, loss 165.91921997070312
Epoch 8, loss 162.89862060546875
Epoch 9, loss 156.0592803955078
Epoch 10, loss 152.92958068847656
Epoch 11, loss 149.87413024902344
Epoch 12, loss 142.90647888183594
Epoch 13, loss 141.7327880859375
Epoch 14, loss 142.99575805664062
Epoch 15, loss 148.14590454101562
Epoch 16, loss 137.66015625
Epoch 17, loss 132.77113342285156
Epoch 18, loss 140.1815185546875
Epoch 19, loss 139.7767333984375
Epoch 20, loss 146.4058837890625
Epoch 21, loss 145.33950805664062
Epoch 22, loss 168.84112548828125
Epoch 23, loss 150.09544372558594
Epoch 24, loss 156.44705200195312
Epoch 25, loss 133.94271850585938
Epoch 26, loss 135.8110809326172
Epoch 27, loss 123.61334228515625
Epoch 28, loss 136.90579223632812
Epoch 29, loss 143.346435546875
Epoch 30, loss 131.543762207

FileNotFoundError: ignored